In [ ]:
import networkx as nx
import numpy as np 

In [45]:
remove_fraction = 0.2
attribute = "gender"
max_iter = 100
eps = 1e-3
none_value = 0
alpha = 0.5
data = "data/Caltech36.gml"


graph = nx.read_gml(data)
graph_original = nx.read_gml(data)

nodes = list(graph.nodes())
np.random.shuffle(nodes)
removed = []
for node in nodes[:int(len(nodes) * remove_fraction)]:
    if attribute in graph.nodes[node]:
        removed.append(node)
        graph.nodes[node][attribute] = none_value

a = nx.adjacency_matrix(graph)
d_inv_sqrt = np.diag([1/np.sqrt(x) if x != 0 else 0 for x in np.sum(a, axis=1)])
s = d_inv_sqrt @ a @ d_inv_sqrt

y_init = np.array([[1 if graph.nodes[node][attribute] == 1 else 0, 1 if graph.nodes[node][attribute] == 2 else 0] for node in graph])
y = y_init.copy()

for _ in range(max_iter):
    y_new = alpha * s @ y + (1 - alpha) * y_init
    if np.linalg.norm(y - y_new, 1) < eps:  
        break
    y = y_new

for idx, node in enumerate(graph.nodes()):
    graph.nodes[node][attribute] = np.argmax(y[idx]) + 1

success, failures = 0, 0
for node in graph:
    if node in removed:
        x, y = graph_original.nodes[node]['gender'], graph.nodes[node]['gender']
        if x != 0:
            if x == y:
                success += 1
            else:
                failures += 1

print(success / (success + failures))

0.678082191780822


gender 0.3 0.65
0.4 0.64
0.1 0.68
0.2 0.65